# Imports

In [1]:
import pandas as pd
import numpy as np
import pickle
import math
import datetime as dt
import json
import requests
from tqdm import tqdm
import os

In [2]:
import geopandas as gpd
import geojson
import h3
import osm2geojson
import shapely
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString
from geopy.distance import geodesic, great_circle

In [3]:
import plotly.graph_objects as go
import plotly.express as px

In [4]:
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

In [5]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 300

In [6]:
import warnings
warnings.simplefilter('ignore')

# Login to DWH

In [7]:
from tochkaml.db.dwh import connect_to_dwh

In [8]:
dwh_login = 'BOCHKAREV_DV'

In [9]:
dwh_pass = 'r5eN3Y6z'

In [10]:
dwh_connection = connect_to_dwh(dwh_login, dwh_pass)

Using 10.103.40.226


In [8]:
from tochkaml import storage

In [9]:
path = '/home/jovyan/work'

# Region

In [24]:
regions = [
    '50', # Москва
    '23', # Краснодар
    '66', # Екатеринбург
    '61', # Ростов-на-Дону
    '02', # Уфа
    '16', # Казань
    '72', # Тюмень
    '74', # Челябинск
    '52', # Нижний Новгород
    '63', # Самара
    '24', # Красноярск
    '26', # Ставрополь
    '54', # Новосибирск
    '42', # Кемерово
    '59', # Пермь
    '34', # Волгоград
    '64', # Саратов
    '38', # Иркутск
    '22', # Барнаул
    '36', # Воронеж
    '56', # Оренбург
    '55', # Омск
    '25', # Владивосток
    '47', # Санкт-Петербург
    '86', # Ханты-Мансийск
    '31', # Белгород
    '18', # Ижевск
    '71', # Тула
    '33', # Владимир
    '58', # Пенза
    '27', # Хабаровск
    '69', # Тверь
    '43', # Киров
    '76', # Ярославль
    '73', # Ульяновск
    '21', # Чебоксары
    '32', # Брянск
    '35', # Вологда
    '29', # Архангельск
    '48', # Липецк
    '62', # Рязань
    '46', # Курск
    '75', # Чита
    '70', # Томск
    '68', # Тамбов
    '37', # Иваново
    '30', # Астрахань
    '40', # Калуга
    '03', # Улан-Удэ
    '39', # Калининград
    '67', # Смоленск
    '14', # Якутск
    '45', # Курган
    '11', # Сыктывкар
    '28', # Благовещенск
    '13', # Саранск
    '51', # Мурманск
    '57', # Орёл
    '12', # Йошкар-Ола
    '44', # Кострома
    '60', # Псков
    '10', # Петрозаводск
    '53', # Великий Новгород
    '89', # Салехард
    '19', # Абакан
    '65', # Южно-Сахалинск
    '41', # Петропавловск-Камчатский
    '17', # Кызыл
    '08', # Элиста
    '04', # Горно-Алтайск
    '79', # Биробиджан
    '49', # Магадан
    '87', # Анадырь
    # '83', # Нарьян-Мар
]

In [25]:
len(regions)

73

# EGRUL

In [14]:
def get_df_inn_region(region):
    df_inn_region = pd.read_sql_query(f"""
        SELECT
              E.inn
            , E.tax_agency_code
            , E.begin_date
            , E.end_date
            , E.end_code
            , E.full_address
            , E.kladr_code
            , E.main_okved

            , ERUZ.inn inn_eruz
            , ERUZ.start_reg_date
            , ERUZ.end_reg_date

            ,D.year
            ,CASE WHEN D.id IS NULL THEN -1 ELSE D.correction_num END correction_num

            ,COALESCE(
                B.assets_balance_fiscal_period,
                BS.assets_balance_fiscal_period,
                0
                )
                *
                CASE
                    WHEN D.okei = 383 THEN 1
                    WHEN D.okei = 384 THEN 1e3
                    WHEN D.okei = 385 THEN 1e6
                    ELSE 1
                END
                assets_balance

            ,COALESCE(
                B.short_borrowed_funds_fiscal_period,
                BS.short_borrowed_funds_fiscal_period,
                0
                )
                *
                CASE
                    WHEN D.okei = 383 THEN 1
                    WHEN D.okei = 384 THEN 1e3
                    WHEN D.okei = 385 THEN 1e6
                    ELSE 1
                END
                short_borrowed_funds

            ,COALESCE(
                B.long_borrowed_funds_fiscal_period,
                BS.long_borrowed_funds_fiscal_period,
                0
                )
                *
                CASE
                    WHEN D.okei = 383 THEN 1
                    WHEN D.okei = 384 THEN 1e3
                    WHEN D.okei = 385 THEN 1e6
                    ELSE 1
                END
                long_borrowed_funds

            ,COALESCE(
                FR.revenue_fiscal_year,
                FRS.revenue_fiscal_year,
                0
                )
                *
                CASE
                    WHEN D.okei = 383 THEN 1
                    WHEN D.okei = 384 THEN 1e3
                    WHEN D.okei = 385 THEN 1e6
                    ELSE 1
                END
                revenue

            ,COALESCE(
                FR.net_profit_loss_fiscal_year,
                FRS.net_profit_loss_fiscal_year,
                0
                )
                *
                CASE
                    WHEN D.okei = 383 THEN 1
                    WHEN D.okei = 384 THEN 1e3
                    WHEN D.okei = 385 THEN 1e6
                    ELSE 1
                END
                net_profit_loss

        FROM
            DATAMART.EGRUL E

            LEFT JOIN (
                SELECT
                      inn
                    , MAX(start_reg_date) start_reg_date
                    , MAX(end_reg_date) end_reg_date
                FROM CORE.ESB_ERUZ_CONTRACTOR
                GROUP BY inn
                ) ERUZ
                ON E.inn = ERUZ.inn

            LEFT JOIN STAGE.GIRBO_TAX_PAYER TP
                ON E.inn = TP.org_inn
            LEFT JOIN STAGE.GIRBO_DOCUMENT D
                ON TP.id = D.id AND D.year = 2020 AND D.period = 34
            LEFT JOIN STAGE.GIRBO_BALANCE B
                ON TP.id = B.id
            LEFT JOIN STAGE.GIRBO_FINANCIAL_RESULT FR
                ON TP.id = FR.id
            LEFT JOIN STAGE.GIRBO_BALANCE_SIMPLE BS
                ON TP.id = BS.id
            LEFT JOIN STAGE.GIRBO_FINANCIAL_RESULT_SIMPLE FRS
                ON TP.id = FRS.id  

        WHERE
            1 = 1
            AND E.kind = 1
            AND SUBSTR(E.inn, 0, 2) = '{region}'
            AND SUBSTR(E.tax_agency_code, 0, 2) = '{region}'
            AND EXTRACT(YEAR FROM E.begin_date) <= 2021
            AND NVL(EXTRACT(YEAR FROM E.end_date),2099) >= 2020
        """,
        dwh_connection)
    
    df_inn_region = df_inn_region.sort_values(
        by='correction_num',
        axis=0,
        ascending=True
    ).drop_duplicates(
        subset='inn',
        keep='last',
        ignore_index=True)
    
    return df_inn_region

# Update FIAS from Karma

In [15]:
api_url = 'https://karma-egrul-http.query.consul-test/api/v1/jsonrpc'

In [16]:
def f_get_data(inn):
    return {
        'jsonrpc':'2.0',
        'method':'get_company',
        'params':{'inn':inn},
        'id':1
    }

In [17]:
def f_parse_egrul(egrul_json):
    
    res = egrul_json['result']['jsondata']
    result = {}
        
    # ЮЛ    
    if 'СвЮЛ' in res:
        result['inn'] = res['СвЮЛ']['ИНН'] if 'ИНН' in res['СвЮЛ'] else None

        # адрес
        if 'СвАдресЮЛ' in res['СвЮЛ']:
            if 'АдресРФ' in res['СвЮЛ']['СвАдресЮЛ']:
                result['kladr_code'] = res['СвЮЛ']['СвАдресЮЛ']['АдресРФ']['КодАдрКладр'] if 'КодАдрКладр' in res['СвЮЛ']['СвАдресЮЛ']['АдресРФ'] else None

            if 'СвАдрЮЛФИАС' in res['СвЮЛ']['СвАдресЮЛ']:
                result['fias_code'] = res['СвЮЛ']['СвАдресЮЛ']['СвАдрЮЛФИАС']['ИдНом'] if 'ИдНом' in res['СвЮЛ']['СвАдресЮЛ']['СвАдрЮЛФИАС'] else None

    return result

# df_inn

In [ ]:
for reg in regions:
    
    print('--------------------------------')
    print(reg)
    
    try:
        storage.personal.load(f'df_inn_{str(reg)}_2.csv.gz')
        print('already exists')
        continue
    except:
        pass
    
    print("begin")
    
    df_inn = get_df_inn_region(reg)
    
    print("finish select")
    print(dt.datetime.now())
    
    df_inn['count_2020'] = 0
    df_inn.loc[
        (df_inn.begin_date.dt.year <= 2020) &
        ((df_inn.end_date.isnull()) | (df_inn.end_date.dt.year >= 2020)),
        'count_2020'
    ] = 1
    

    df_inn['count_2021'] = 0
    df_inn.loc[
        (df_inn.begin_date.dt.year <= 2021) &
        ((df_inn.end_date.isnull()) | (df_inn.end_date.dt.year >= 2021)),
        'count_2021'
    ] = 1

    df_inn['begin_2021'] = 0
    df_inn.loc[
        (df_inn.begin_date.dt.year == 2021),
        'begin_2021'
    ] = 1

    df_inn['end_2021'] = 0
    df_inn.loc[
        (df_inn.end_date.dt.year == 2021),
        'end_2021'
    ] = 1

    print(df_inn.groupby([df_inn.inn.map(len), df_inn.full_address.notnull(), df_inn.kladr_code.notnull()]).size())
    
    print('begin update from karma')
    print(dt.datetime.now())
    
    df_inn_upd = pd.DataFrame(columns=[
        'inn',
        'kladr_code',
        'fias_code'
    ])
    
    for inn in tqdm(df_inn.loc[df_inn.kladr_code.isnull(), 'inn']):
        try:
            response = requests.post(
                api_url,
                json=f_get_data(inn),
                headers={'content-type':'application/json'},
                verify=False
            )
            egrul_json = response.json()
            df_inn_upd = pd.concat(
                [
                    df_inn_upd,
                    pd.DataFrame(f_parse_egrul(egrul_json), index=[0])],
                ignore_index=True
            ) 
        except Exception:
            pass
    
    df_inn = df_inn.merge(
        df_inn_upd[['inn', 'fias_code']],
        how = 'left',
        on = 'inn'
    )
    
    print(df_inn.groupby(
        [
            df_inn.inn.map(len),
            df_inn.full_address.notnull(),
            df_inn.kladr_code.notnull(),
            df_inn.fias_code.notnull()
        ]
    ).size())
    
    df_inn['kladr_11'] = df_inn['kladr_code'].str[:11]
    
    print("finish update")
    
    df_inn.to_csv(f'/home/jovyan/work/df_inn_{str(reg)}_2.csv.gz', index=False, sep=';', compression='gzip')
    storage.personal.save(f'df_inn_{str(reg)}_2.csv.gz')
    
    print(f"finish saving {reg}")
    
    df_kladr = pd.DataFrame(df_inn.loc[
        (df_inn.inn.map(len)==10) &
        (df_inn.kladr_code.notnull()) &
        (df_inn.kladr_code.str[:2]==f'{reg}'),
        'kladr_11'
    ].unique())
    
    df_kladr.rename(columns={0:'kladr_11'}, inplace=True)
    
    df_kladr.to_csv(f'/home/jovyan/work/df_kladr_{reg}.csv.gz', index=False, compression='gzip')
    storage.personal.save(f'df_kladr_{reg}.csv.gz')
    
    print(f"finish saving kladr {reg}")
    
    df_fias = pd.DataFrame(df_inn.loc[
        df_inn.fias_code.notnull(),
        'fias_code'
    ].unique())
    
    df_fias.rename(columns={0:'fias_code'}, inplace=True)
    
    df_fias.to_csv(f'/home/jovyan/work/df_fias_{reg}.csv.gz', index=False, compression='gzip')
    storage.personal.save(f'df_fias_{reg}.csv.gz')
    
    print(f"finish saving fias {reg}")
    
    print(f"finish {reg}")
    print('--------------------------------')
    

--------------------------------
70
2022-11-22 01:54.48 [info     ] check for directory            fs=s3fs path=df_inn_70_2.csv.gz
2022-11-22 01:54.48 [info     ] load file                      path=df_inn_70_2.csv.gz
2022-11-22 01:54.48 [info     ] open file for read             fs=s3fs path=df_inn_70_2.csv.gz
begin
finish select
2022-11-22 01:55:39.591657
inn  full_address  kladr_code
10   True          False          3493
                   True          25155
dtype: int64
begin update from karma
2022-11-22 01:55:39.640645


  9%|▉         | 323/3493 [00:25<04:16, 12.38it/s]

In [ ]:
# get kladr and fias files

path = '/home/jovyan/work'

for reg in []:

    print('--------------------------------')
    print(reg)
    
    storage.personal.load(f'df_inn_{str(reg)}_2.csv.gz')
    
    storage.personal.load(f'df_inn_{reg}_2.csv.gz')
    df_inn = pd.read_csv(
        f'{path}/df_inn_{reg}_2.csv.gz',
        parse_dates=['begin_date','end_date','start_reg_date','end_reg_date'],
        dtype={
            'inn':str,
            'tax_agency_code':str,
            'end_code':str,
            'kladr_code':str,
            'kladr_11':str,
            'inn_eruz':str
        },
        sep=';',
        compression='gzip'
    )
    
    print(df_inn.groupby(
        [
            df_inn.inn.map(len),
            df_inn.full_address.notnull(),
            df_inn.kladr_code.notnull(),
            df_inn.fias_code.notnull()
        ]
    ).size())
    
    df_inn['kladr_11'] = df_inn['kladr_code'].str[:11]
    
#     df_kladr = pd.DataFrame(df_inn.loc[
#         (df_inn.inn.map(len)==10) &
#         (df_inn.kladr_code.notnull()) &
#         (df_inn.kladr_code.str[:2]==f'{reg}'),
#         'kladr_11'
#     ].unique())
    
#     df_kladr.rename(columns={0:'kladr_11'}, inplace=True)
    
#     df_kladr.to_csv(f'/home/jovyan/work/df_kladr_{reg}.csv.gz', index=False, compression='gzip')
#     storage.personal.save(f'df_kladr_{reg}.csv.gz')
    
#     print(f"finish saving kladr {reg}")

    df_inn_upd = pd.DataFrame(columns=[
        'inn',
        'kladr_code',
        'fias_code'
    ])
    
    for inn in tqdm(df_inn.loc[df_inn.kladr_code.isnull(), 'inn']):
        try:
            response = requests.post(
                api_url,
                json=f_get_data(inn),
                headers={'content-type':'application/json'},
                verify=False
            )
            egrul_json = response.json()
            df_inn_upd = pd.concat(
                [
                    df_inn_upd,
                    pd.DataFrame(f_parse_egrul(egrul_json), index=[0])],
                ignore_index=True
            ) 
        except Exception:
            pass
    
    
    df_inn = df_inn.drop(columns=['fias_code']).merge(
        df_inn_upd[['inn', 'fias_code']],
        how = 'left',
        on = 'inn'
    )
    
    print(df_inn.groupby(
        [
            df_inn.inn.map(len),
            df_inn.full_address.notnull(),
            df_inn.kladr_code.notnull(),
            df_inn.fias_code.notnull()
        ]
    ).size())
    
    df_inn.to_csv(f'/home/jovyan/work/df_inn_{str(reg)}_2.csv.gz', index=False, sep=';', compression='gzip')
    storage.personal.save(f'df_inn_{str(reg)}_2.csv.gz')
    
    print(f"finish saving {reg}")
    
    df_fias = pd.DataFrame(df_inn.loc[
        df_inn.fias_code.notnull(),
        'fias_code'
    ].unique())
    
    df_fias.rename(columns={0:'fias_code'}, inplace=True)
    
    df_fias.to_csv(f'/home/jovyan/work/df_fias_{reg}.csv.gz', index=False, compression='gzip')
    storage.personal.save(f'df_fias_{reg}.csv.gz')
    
    print(f"finish saving fias {reg}")

    print('--------------------------------')
    

# Hexes

In [22]:
h3_resolution = 5

In [20]:
for reg in regions:
    
    print('--------------------------------')
    print(reg)
    print(dt.datetime.now())
    
    try:
        storage.personal.load(f'hexes_{reg}_ul_2.parquet.gzip')
        print('already exists')
        continue
    except:
        pass
    
    try:
        storage.personal.load(f'{int(reg)}.parquet.gzip')
        hexes_gdf = gpd.read_parquet(f'{path}/{int(reg)}.parquet.gzip')
    except:
        print('no hexes file')
        continue
    
    try:
        storage.personal.load(f'df_kladr_{reg}_upd.csv.gz')
        df_kladr_upd = pd.read_csv(
            f'{path}/df_kladr_{reg}_upd.csv.gz',
            sep=';',
            compression='gzip',
            dtype={'kladr_11':str, 'kladr':str}
        )
    except:
        print('no kladr file')
        continue
    
    try:
        storage.personal.load(f'df_fias_{reg}_upd.csv.gz')
        df_fias_upd = pd.read_csv(
            f'{path}/df_fias_{reg}_upd.csv.gz',
            sep=';',
            compression='gzip'
        )
    except:
        print('no fias file')
        continue
    
    try:
        storage.personal.load(f'df_inn_{reg}_2.csv.gz')
        df_inn = pd.read_csv(
            f'{path}/df_inn_{reg}_2.csv.gz',
            parse_dates=['begin_date','end_date','start_reg_date','end_reg_date'],
            dtype={
                'inn':str,
                'tax_agency_code':str,
                'end_code':str,
                'kladr_code':str,
                'kladr_11':str,
                'inn_eruz':str
            },
            sep=';',
            compression='gzip'
        )
    except:
        print('no inn file')
        continue
    
    df_inn['kladr_11'] = df_inn['kladr_code'].str[:11]
    df_inn = df_inn.merge(
        df_kladr_upd[['kladr_11', 'latitude', 'longitude']],
        how='left',
        on='kladr_11'
    )
    
    df_inn = df_inn.merge(
        df_fias_upd[['fias_code', 'latitude', 'longitude']].rename(columns={'latitude':'latitude_fias', 'longitude':'longitude_fias'}),
        how='left',
        on='fias_code'
    )
    
    df_inn.loc[df_inn.latitude.isnull(), 'latitude'] = df_inn['latitude_fias']
    df_inn.loc[df_inn.longitude.isnull(), 'longitude'] = df_inn['longitude_fias']
    df_inn.drop(columns=['latitude_fias', 'longitude_fias'], inplace=True)
    
    df_inn[f'h3_id_{h3_resolution}'] = None
    for index, row in df_inn.iterrows():
        h3_id = h3.geo_to_h3(
            lat = row['latitude'], 
            lng = row['longitude'], 
            resolution = h3_resolution
        )
        df_inn.loc[index, f'h3_id_{h3_resolution}'] = h3_id
    
    df_inn_gr = df_inn[[
        'assets_balance',
        'short_borrowed_funds',
        'long_borrowed_funds',
        'revenue',
        'net_profit_loss',
        'count_2020',
        'count_2021',
        'begin_2021',
        'end_2021',
        f'h3_id_{h3_resolution}'
    ]].groupby(f'h3_id_{h3_resolution}', as_index=False).sum()
    
    hexes_gdf = hexes_gdf[['geometry', 'hex_id']].merge(
        df_inn_gr,
        how='left',
        left_on = 'hex_id',
        right_on = f'h3_id_{h3_resolution}'
    )
    
    hexes_gdf.fillna(0, inplace=True)
    
    hexes_gdf['begin_2021_rel'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.count_2021 == 0) &
        (hexes_gdf.begin_2021 == 0),
        'begin_2021_rel'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.count_2021 == 0) &
        (hexes_gdf.begin_2021 > 0),
        'begin_2021_rel'
    ] = -2
    hexes_gdf.loc[
        (hexes_gdf.count_2021 > 0),
        'begin_2021_rel'
    ] = hexes_gdf['begin_2021'] / hexes_gdf['count_2021']

    hexes_gdf['end_2021_rel'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.count_2021 == 0) &
        (hexes_gdf.end_2021 == 0),
        'end_2021_rel'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.count_2021 > 0),
        'end_2021_rel'
    ] = hexes_gdf['end_2021'] / hexes_gdf['count_2021']

    hexes_gdf['count_2021_2020_rel'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.count_2020 == 0) &
        (hexes_gdf.count_2021 == 0),
        'count_2021_2020_rel'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.count_2020 == 0) &
        (hexes_gdf.count_2021 > 0),
        'count_2021_2020_rel'
    ] = -2
    hexes_gdf.loc[
        (hexes_gdf.count_2020 > 0) &
        (hexes_gdf.count_2021 == 0),
        'count_2021_2020_rel'
    ] = -1
    hexes_gdf.loc[
        (hexes_gdf.count_2020 > 0) &
        (hexes_gdf.count_2021 > 0),
        'count_2021_2020_rel'
    ] = hexes_gdf['count_2021'] / hexes_gdf['count_2020']

    hexes_gdf['net_profit_loss_to_revenue_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.revenue == 0),
        'net_profit_loss_to_revenue_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.revenue > 0),
        'net_profit_loss_to_revenue_2020'
    ] = hexes_gdf['net_profit_loss'] / hexes_gdf['revenue']

    hexes_gdf['net_profit_loss_to_assets_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.assets_balance == 0),
        'net_profit_loss_to_assets_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.assets_balance > 0),
        'net_profit_loss_to_assets_2020'
    ] = hexes_gdf['net_profit_loss'] / hexes_gdf['assets_balance']

    hexes_gdf['short_borrowed_funds_to_revenue_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.revenue == 0),
        'short_borrowed_funds_to_revenue_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.revenue > 0),
        'short_borrowed_funds_to_revenue_2020'
    ] = hexes_gdf['short_borrowed_funds'] / hexes_gdf['revenue']

    hexes_gdf['long_borrowed_funds_to_revenue_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.revenue == 0),
        'long_borrowed_funds_to_revenue_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.revenue > 0),
        'long_borrowed_funds_to_revenue_2020'
    ] = hexes_gdf['long_borrowed_funds'] / hexes_gdf['revenue']

    hexes_gdf['revenue_to_assets_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.assets_balance == 0),
        'revenue_to_assets_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.assets_balance > 0),
        'revenue_to_assets_2020'
    ] = hexes_gdf['revenue'] / hexes_gdf['assets_balance']

    hexes_gdf['assets_to_count_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.count_2020 == 0),
        'assets_to_count_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.count_2020 > 0),
        'assets_to_count_2020'
    ] = hexes_gdf['assets_balance'] / hexes_gdf['count_2020']

    hexes_gdf['net_profit_loss_to_count_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.count_2020 == 0),
        'net_profit_loss_to_count_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.count_2020 > 0),
        'net_profit_loss_to_count_2020'
    ] = hexes_gdf['net_profit_loss'] / hexes_gdf['count_2020']

    hexes_gdf['revenue_to_count_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.count_2020 == 0),
        'revenue_to_count_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.count_2020 > 0),
        'revenue_to_count_2020'
    ] = hexes_gdf['revenue'] / hexes_gdf['count_2020']
    
    hexes_gdf.drop(columns=['h3_id_5']).to_parquet(
        f'home/jovyan/work/hexes_{reg}_ul_2.parquet.gzip',
        compression='gzip',
        index=False,
        engine='pyarrow' 
    )
    storage.personal.save(f'hexes_{reg}_ul_2.parquet.gzip')
    
    print('!!! SUCСESS !!!')


--------------------------------
50
2022-11-22 01:24:27.411749
2022-11-22 01:24.27 [info     ] check for directory            fs=s3fs path=hexes_50_ul_2.parquet.gzip
2022-11-22 01:24.27 [info     ] load file                      path=hexes_50_ul_2.parquet.gzip
2022-11-22 01:24.27 [info     ] open file for read             fs=s3fs path=hexes_50_ul_2.parquet.gzip
2022-11-22 01:24.27 [info     ] check for directory            fs=s3fs path=50.parquet.gzip
2022-11-22 01:24.27 [info     ] load file                      path=50.parquet.gzip
2022-11-22 01:24.27 [info     ] open file for read             fs=s3fs path=50.parquet.gzip
2022-11-22 01:24.27 [info     ] open file for write            fs=localfs path=50.parquet.gzip
2022-11-22 01:24.27 [info     ] check for directory            fs=s3fs path=df_kladr_50_upd.csv.gz
2022-11-22 01:24.27 [info     ] load file                      path=df_kladr_50_upd.csv.gz
2022-11-22 01:24.27 [info     ] open file for read             fs=s3fs path=df_klad

In [26]:
# update h3

for reg in regions:
    
    print('--------------------------------')
    print(reg)
    print(dt.datetime.now())
    
    try:
        storage.personal.load(f'df_kladr_{reg}_upd.csv.gz')
        df_kladr_upd = pd.read_csv(
            f'{path}/df_kladr_{reg}_upd.csv.gz',
            sep=';',
            compression='gzip',
            dtype={'kladr_11':str, 'kladr':str}
        )
    except:
        print('no kladr file')
        continue
    
    try:
        storage.personal.load(f'df_fias_{reg}_upd.csv.gz')
        df_fias_upd = pd.read_csv(
            f'{path}/df_fias_{reg}_upd.csv.gz',
            sep=';',
            compression='gzip'
        )
    except:
        print('no fias file')
        continue
    
    try:
        storage.personal.load(f'df_inn_{reg}_2.csv.gz')
        df_inn = pd.read_csv(
            f'{path}/df_inn_{reg}_2.csv.gz',
            parse_dates=['begin_date','end_date','start_reg_date','end_reg_date'],
            dtype={
                'inn':str,
                'tax_agency_code':str,
                'end_code':str,
                'kladr_code':str,
                'kladr_11':str,
                'inn_eruz':str
            },
            sep=';',
            compression='gzip'
        )
    except:
        print('no inn file')
        continue
    
    df_inn['kladr_11'] = df_inn['kladr_code'].str[:11]
    df_inn = df_inn.merge(
        df_kladr_upd[['kladr_11', 'latitude', 'longitude']],
        how='left',
        on='kladr_11'
    )
    
    df_inn = df_inn.merge(
        df_fias_upd[['fias_code', 'latitude', 'longitude']].rename(columns={'latitude':'latitude_fias', 'longitude':'longitude_fias'}),
        how='left',
        on='fias_code'
    )
    
    df_inn.loc[df_inn.latitude.isnull(), 'latitude'] = df_inn['latitude_fias']
    df_inn.loc[df_inn.longitude.isnull(), 'longitude'] = df_inn['longitude_fias']
    df_inn.drop(columns=['latitude_fias', 'longitude_fias'], inplace=True)
    
    df_inn[f'h3_id_{h3_resolution}'] = None
    for index, row in df_inn.iterrows():
        h3_id = h3.geo_to_h3(
            lat = row['latitude'], 
            lng = row['longitude'], 
            resolution = h3_resolution
        )
        df_inn.loc[index, f'h3_id_{h3_resolution}'] = h3_id
    
    df_inn.to_csv(f'{path}/df_inn_{str(reg)}_2_upd.csv.gz', index=False, sep=';', compression='gzip')
    storage.personal.save(f'df_inn_{str(reg)}_2_upd.csv.gz')
    
    print('!!! SUCСESS !!!')


--------------------------------
50
2022-11-22 15:47:31.317418
2022-11-22 15:47.31 [info     ] check for directory            fs=s3fs path=df_kladr_50_upd.csv.gz
2022-11-22 15:47.31 [info     ] load file                      path=df_kladr_50_upd.csv.gz
2022-11-22 15:47.31 [info     ] open file for read             fs=s3fs path=df_kladr_50_upd.csv.gz
2022-11-22 15:47.31 [info     ] open file for write            fs=localfs path=df_kladr_50_upd.csv.gz
2022-11-22 15:47.31 [info     ] check for directory            fs=s3fs path=df_fias_50_upd.csv.gz
2022-11-22 15:47.31 [info     ] load file                      path=df_fias_50_upd.csv.gz
2022-11-22 15:47.31 [info     ] open file for read             fs=s3fs path=df_fias_50_upd.csv.gz
2022-11-22 15:47.31 [info     ] open file for write            fs=localfs path=df_fias_50_upd.csv.gz
2022-11-22 15:47.31 [info     ] check for directory            fs=s3fs path=df_inn_50_2.csv.gz
2022-11-22 15:47.31 [info     ] load file                      p

# Hexes with Neighbours

In [29]:
storage.personal.load('spr_regions_neighbours.xlsx')

2022-11-23 14:00.32 [info     ] check for directory            fs=s3fs path=spr_regions_neighbours.xlsx
2022-11-23 14:00.32 [info     ] load file                      path=spr_regions_neighbours.xlsx
2022-11-23 14:00.32 [info     ] open file for read             fs=s3fs path=spr_regions_neighbours.xlsx
2022-11-23 14:00.32 [info     ] open file for write            fs=localfs path=spr_regions_neighbours.xlsx


In [13]:
spr_neighbours = pd.read_excel(
    f'{path}/spr_regions_neighbours.xlsx',
    sheet_name='spr_regions_neighbours',
    dtype={
        'touches':bool,
        'intersects':bool,
        'is_neighbour':bool,
        'region_code_1':str,
        'region_code_2':str}
)

In [14]:
spr_neighbours.head()

,touches,intersects,is_neighbour,region_name_1,region_code_1,region_name_2,region_code_2
0,False,True,True,Адыгея,1,Адыгея,1
1,False,False,False,Адыгея,1,Алтай,4
2,False,False,False,Адыгея,1,Алтайский,22
3,False,False,False,Адыгея,1,Амурская,28
4,False,False,False,Адыгея,1,Архангельская,29


In [15]:
spr_neighbours.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7225 entries, 0 to 7224
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   touches        7225 non-null   bool  
 1   intersects     7225 non-null   bool  
 2   is_neighbour   7225 non-null   bool  
 3   region_name_1  7225 non-null   object
 4   region_code_1  7225 non-null   object
 5   region_name_2  7225 non-null   object
 6   region_code_2  7225 non-null   object
dtypes: bool(3), object(4)
memory usage: 247.1+ KB


In [16]:
spr_neighbours['region_code_1'] = spr_neighbours['region_code_1'].apply(lambda x: f'0{x}' if len(x)==1 else x)

In [17]:
spr_neighbours['region_code_2'] = spr_neighbours['region_code_2'].apply(lambda x: f'0{x}' if len(x)==1 else x)

In [18]:
spr_neighbours.head()

,touches,intersects,is_neighbour,region_name_1,region_code_1,region_name_2,region_code_2
0,False,True,True,Адыгея,01,Адыгея,01
1,False,False,False,Адыгея,01,Алтай,04
2,False,False,False,Адыгея,01,Алтайский,22
3,False,False,False,Адыгея,01,Амурская,28
4,False,False,False,Адыгея,01,Архангельская,29


In [19]:
spr_neighbours[spr_neighbours['is_neighbour']==True].head()

,touches,intersects,is_neighbour,region_name_1,region_code_1,region_name_2,region_code_2
0,False,True,True,Адыгея,01,Адыгея,01
31,True,True,True,Адыгея,01,Краснодарский,23
86,False,True,True,Алтай,04,Алтай,04
87,True,True,True,Алтай,04,Алтайский,22
112,True,True,True,Алтай,04,Кемеровская,42


In [20]:
regions_bad = [
    '77',
    '78',
    '05',
    '91',
    '20',
    '07',
    '15',
    '09',
    '06',
    '01',
    '92',
    '83'
]

In [23]:
# with neighbours

for reg in regions:
    
    print('--------------------------------')
    print(reg)
    print(dt.datetime.now())
    
    try:
        storage.personal.load(f'hexes_{reg}_ul_3.parquet.gzip')
        print('already exists')
        continue
    except:
        pass
    
    try:
        storage.personal.load(f'{int(reg)}.parquet.gzip')
        hexes_gdf = gpd.read_parquet(f'{path}/{int(reg)}.parquet.gzip')
    except:
        print('no hexes file')
        continue
    
    reg_neighbours = list(
        spr_neighbours.loc[
            (spr_neighbours['region_code_1'] == reg) &
            (spr_neighbours['is_neighbour'] == True),
            'region_code_2'
        ]
    )
    reg_neighbours = [r for r in reg_neighbours if r not in regions_bad]
    print(f'neighbours: {reg_neighbours}')
    print(f'neighbours count: {len(reg_neighbours)}')
    
    try:
        df_inn_neighbours = {}
        
        for reg_n in reg_neighbours:
    
            storage.personal.load(f'df_inn_{reg_n}_2_upd.csv.gz')
            df_inn_neighbours[reg_n] = pd.read_csv(
                f'{path}/df_inn_{reg_n}_2_upd.csv.gz',
                parse_dates=['begin_date','end_date','start_reg_date','end_reg_date'],
                dtype={
                    'inn':str,
                    'tax_agency_code':str,
                    'end_code':str,
                    'kladr_code':str,
                    'kladr_11':str,
                    'inn_eruz':str
                },
                sep=';',
                compression='gzip'
            )
          
    except:
        print('no inn files')
        continue
    
    df_inn_n = pd.DataFrame()
    for reg_n in reg_neighbours:
        df_inn_n = pd.concat(
            [df_inn_n, df_inn_neighbours[reg_n]],
            axis=0,
            join='outer',
            ignore_index=True
        )
    print('finish concat')
    print(dt.datetime.now())
    
    df_inn_n_gr = df_inn_n[[
        'assets_balance',
        'short_borrowed_funds',
        'long_borrowed_funds',
        'revenue',
        'net_profit_loss',
        'count_2020',
        'count_2021',
        'begin_2021',
        'end_2021',
        f'h3_id_{h3_resolution}'
    ]].groupby(f'h3_id_{h3_resolution}', as_index=False).sum()
    
    hexes_gdf = hexes_gdf[['geometry', 'hex_id']].merge(
        df_inn_n_gr,
        how='left',
        left_on = 'hex_id',
        right_on = f'h3_id_{h3_resolution}'
    )
    
    hexes_gdf.fillna(0, inplace=True)
    
    hexes_gdf['begin_2021_rel'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.count_2021 == 0) &
        (hexes_gdf.begin_2021 == 0),
        'begin_2021_rel'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.count_2021 == 0) &
        (hexes_gdf.begin_2021 > 0),
        'begin_2021_rel'
    ] = -2
    hexes_gdf.loc[
        (hexes_gdf.count_2021 > 0),
        'begin_2021_rel'
    ] = hexes_gdf['begin_2021'] / hexes_gdf['count_2021']

    hexes_gdf['end_2021_rel'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.count_2021 == 0) &
        (hexes_gdf.end_2021 == 0),
        'end_2021_rel'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.count_2021 > 0),
        'end_2021_rel'
    ] = hexes_gdf['end_2021'] / hexes_gdf['count_2021']

    hexes_gdf['count_2021_2020_rel'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.count_2020 == 0) &
        (hexes_gdf.count_2021 == 0),
        'count_2021_2020_rel'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.count_2020 == 0) &
        (hexes_gdf.count_2021 > 0),
        'count_2021_2020_rel'
    ] = -2
    hexes_gdf.loc[
        (hexes_gdf.count_2020 > 0) &
        (hexes_gdf.count_2021 == 0),
        'count_2021_2020_rel'
    ] = -1
    hexes_gdf.loc[
        (hexes_gdf.count_2020 > 0) &
        (hexes_gdf.count_2021 > 0),
        'count_2021_2020_rel'
    ] = hexes_gdf['count_2021'] / hexes_gdf['count_2020']

    hexes_gdf['net_profit_loss_to_revenue_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.revenue == 0),
        'net_profit_loss_to_revenue_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.revenue > 0),
        'net_profit_loss_to_revenue_2020'
    ] = hexes_gdf['net_profit_loss'] / hexes_gdf['revenue']

    hexes_gdf['net_profit_loss_to_assets_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.assets_balance == 0),
        'net_profit_loss_to_assets_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.assets_balance > 0),
        'net_profit_loss_to_assets_2020'
    ] = hexes_gdf['net_profit_loss'] / hexes_gdf['assets_balance']

    hexes_gdf['short_borrowed_funds_to_revenue_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.revenue == 0),
        'short_borrowed_funds_to_revenue_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.revenue > 0),
        'short_borrowed_funds_to_revenue_2020'
    ] = hexes_gdf['short_borrowed_funds'] / hexes_gdf['revenue']

    hexes_gdf['long_borrowed_funds_to_revenue_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.revenue == 0),
        'long_borrowed_funds_to_revenue_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.revenue > 0),
        'long_borrowed_funds_to_revenue_2020'
    ] = hexes_gdf['long_borrowed_funds'] / hexes_gdf['revenue']

    hexes_gdf['revenue_to_assets_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.assets_balance == 0),
        'revenue_to_assets_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.assets_balance > 0),
        'revenue_to_assets_2020'
    ] = hexes_gdf['revenue'] / hexes_gdf['assets_balance']

    hexes_gdf['assets_to_count_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.count_2020 == 0),
        'assets_to_count_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.count_2020 > 0),
        'assets_to_count_2020'
    ] = hexes_gdf['assets_balance'] / hexes_gdf['count_2020']

    hexes_gdf['net_profit_loss_to_count_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.count_2020 == 0),
        'net_profit_loss_to_count_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.count_2020 > 0),
        'net_profit_loss_to_count_2020'
    ] = hexes_gdf['net_profit_loss'] / hexes_gdf['count_2020']

    hexes_gdf['revenue_to_count_2020'] = np.nan
    hexes_gdf.loc[
        (hexes_gdf.count_2020 == 0),
        'revenue_to_count_2020'
    ] = -3
    hexes_gdf.loc[
        (hexes_gdf.count_2020 > 0),
        'revenue_to_count_2020'
    ] = hexes_gdf['revenue'] / hexes_gdf['count_2020']
    
    hexes_gdf.drop(columns=['h3_id_5']).to_parquet(
        f'home/jovyan/work/hexes_{reg}_ul_3.parquet.gzip',
        compression='gzip',
        index=False,
        engine='pyarrow' 
    )
    storage.personal.save(f'hexes_{reg}_ul_3.parquet.gzip')
    
    print('!!! SUCСESS !!!')


--------------------------------
14
2022-11-23 15:50:03.464058
2022-11-23 15:50.03 [info     ] check for directory            fs=s3fs path=hexes_14_ul_3.parquet.gzip
2022-11-23 15:50.03 [info     ] load file                      path=hexes_14_ul_3.parquet.gzip
2022-11-23 15:50.03 [info     ] open file for read             fs=s3fs path=hexes_14_ul_3.parquet.gzip
neighbours: ['28', '75', '38', '24', '49', '14', '27', '87']
neighbours count: 8
2022-11-23 15:50.03 [info     ] check for directory            fs=s3fs path=df_inn_28_2_upd.csv.gz
2022-11-23 15:50.03 [info     ] load file                      path=df_inn_28_2_upd.csv.gz
2022-11-23 15:50.03 [info     ] open file for read             fs=s3fs path=df_inn_28_2_upd.csv.gz
2022-11-23 15:50.03 [info     ] open file for write            fs=localfs path=df_inn_28_2_upd.csv.gz
2022-11-23 15:50.04 [info     ] check for directory            fs=s3fs path=df_inn_75_2_upd.csv.gz
2022-11-23 15:50.04 [info     ] load file                      pa

In [24]:
hexes_gdf.head()

,geometry,hex_id,h3_id_5,assets_balance,short_borrowed_funds,long_borrowed_funds,revenue,net_profit_loss,count_2020,count_2021,begin_2021,end_2021,begin_2021_rel,end_2021_rel,count_2021_2020_rel,net_profit_loss_to_revenue_2020,net_profit_loss_to_assets_2020,short_borrowed_funds_to_revenue_2020,long_borrowed_funds_to_revenue_2020,revenue_to_assets_2020,assets_to_count_2020,net_profit_loss_to_count_2020,revenue_to_count_2020
0,"POLYGON ((152.19145 70.14677, 152.46343 70.117...",8504f407fffffff,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,-3.0,-3.0,-3.000000,-3.000000,-3.0,-3.0,-3.00000,-3.000000e+00,-3.000000,-3.0
1,"POLYGON ((112.76913 64.55623, 112.97235 64.585...",850addabfffffff,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,-3.0,-3.0,-3.000000,-3.000000,-3.0,-3.0,-3.00000,-3.000000e+00,-3.000000,-3.0
2,"POLYGON ((106.89310 65.74210, 106.87959 65.650...",850ac577fffffff,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,-3.0,-3.0,-3.000000,-3.000000,-3.0,-3.0,-3.00000,-3.000000e+00,-3.000000,-3.0
3,"POLYGON ((135.68657 62.42413, 135.79428 62.502...",85146513fffffff,85146513fffffff,18257000.0,0.0,0.0,3773000.0,612000.0,7.0,7.0,0.0,0.0,0.0,0.0,1.0,0.162205,0.033521,0.0,0.0,0.20666,2.608143e+06,87428.571429,539000.0
4,"POLYGON ((111.22956 61.03027, 111.37869 60.972...",85152ca3fffffff,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,-3.0,-3.0,-3.000000,-3.000000,-3.0,-3.0,-3.00000,-3.000000e+00,-3.000000,-3.0


In [10]:
storage.personal.load('14.parquet.gzip')        

2022-11-23 15:15.55 [info     ] check for directory            fs=s3fs path=14.parquet.gzip
2022-11-23 15:15.55 [info     ] load file                      path=14.parquet.gzip
2022-11-23 15:15.55 [info     ] open file for read             fs=s3fs path=14.parquet.gzip
2022-11-23 15:15.55 [info     ] open file for write            fs=localfs path=14.parquet.gzip


In [12]:
hexes_gdf = gpd.read_parquet(f'{path}/14.parquet.gzip')

In [52]:
gpd.__version__

'0.12.1'

In [53]:
import pyarrow
pyarrow.__version__

'9.0.0'